# LIBARY

In [5]:
import geopandas as gpd
import pandas as pd
import numpy as np
#import fiona
import pyogrio
import requests
import zipfile
import os
from io import BytesIO
from bs4 import BeautifulSoup

In [6]:
# check if directory exist if not make it, return path
def make_data_path():
    """Create directories for data storage if they do not exist."""
    cwd = os.getcwd()
    parent = os.path.dirname(cwd)
    data_path = os.path.join(parent, 'Data')
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    
    building_data_csv = os.path.join(data_path, 'building_data_csv')
    building_data_gdb = os.path.join(data_path, 'building_data_gdb')
    building_stock_data = os.path.join(data_path, 'building_stock_data')
    if not os.path.exists(building_data_csv):
        os.makedirs(building_data_csv)
    if not os.path.exists(building_data_gdb):
        os.makedirs(building_data_gdb)
    if not os.path.exists(building_stock_data):
        os.makedirs(building_stock_data)

    return building_data_csv, building_data_gdb, building_stock_data

# DOWNLOAD BUILDING DATA

This notebook is to download building data, extract it, then aggregate the data to count the number of building for each census tract.

In [7]:
def fetch_state_links():
    """Fetches state names and their corresponding links from the webpage."""
   
    # URL of the webpage to scrape
    url = "https://disasters.geoplatform.gov/USA_Structures/"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        links = soup.find_all("a", href=True)
        return {link.text.strip(): link["href"] for link in links if "Deliverable" in link["href"]}
    else:
        print("Failed to fetch the webpage. Status code:", response.status_code)
        return {}

def get_link_by_state(state_name, state_links):
    """Returns the link for a given state name."""
    return state_links.get(state_name, "State not found")   

In [8]:
def download_and_extract_zip(state_name, state_links):
    """Downloads and extracts a ZIP file from the given URL.
    Keyword arguments:
    state_name -- Name of the state
    state_links -- Corresponding links for each state
    """
    url = get_link_by_state(state_name, state_links)
    parent_dir = os.path.dirname(os.getcwd())
    output_dir = os.path.join(parent_dir, 'Data', 'building_data_gdb')

    response = requests.get(url, stream=True)
    if response.status_code == 200:
        os.makedirs(output_dir, exist_ok=True)
        zip_path = os.path.join(output_dir, f"{state_name}_Structures.zip")
        
        with open(zip_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(output_dir)
        
        os.remove(zip_path)
        print(f"Downloaded, extracted, and deleted ZIP file for {state_name} to {output_dir}")
    else:
        print("Failed to download the ZIP file.")

In [9]:
state_links = fetch_state_links()
state_links
# download_and_extract_zip("Alaska", state_links)

{'Alabama': 'https://fema-femadata.s3.amazonaws.com/Partners/ORNL/USA_Structures/Alabama/Deliverable202496AL.zip',
 'Alaska': 'https://fema-femadata.s3.amazonaws.com/Partners/ORNL/USA_Structures/Alaska/Deliverable20230728AK.zip',
 'American Samoa': 'https://fema-femadata.s3.amazonaws.com/Partners/ORNL/USA_Structures/American+Samoa/Deliverable20230831AS.zip',
 'Arizona': 'https://fema-femadata.s3.amazonaws.com/Partners/ORNL/USA_Structures/Arizona/Deliverable20230502AZ.zip',
 'Arkansas': 'https://fema-femadata.s3.amazonaws.com/Partners/ORNL/USA_Structures/Arkansas/Deliverable20230630AR.zip',
 'California': 'https://fema-femadata.s3.amazonaws.com/Partners/ORNL/USA_Structures/California/Deliverable20230728CA.zip',
 'Colorado': 'https://fema-femadata.s3.amazonaws.com/Partners/ORNL/USA_Structures/Colorado/Deliverable20230630CO.zip',
 'Connecticut': 'https://fema-femadata.s3.amazonaws.com/Partners/ORNL/USA_Structures/Connecticut/Deliverable20230502CT.zip',
 'Delaware': 'https://fema-femadata.

In [14]:
for state in state_links:
    download_and_extract_zip(state, state_links)

Downloaded, extracted, and deleted ZIP file for Alabama to c:\Users\ysurya\Documents\EarthquakeDamageModel_Heinz\Data\building_data_gdb
Downloaded, extracted, and deleted ZIP file for Alaska to c:\Users\ysurya\Documents\EarthquakeDamageModel_Heinz\Data\building_data_gdb
Downloaded, extracted, and deleted ZIP file for American Samoa to c:\Users\ysurya\Documents\EarthquakeDamageModel_Heinz\Data\building_data_gdb
Downloaded, extracted, and deleted ZIP file for Arizona to c:\Users\ysurya\Documents\EarthquakeDamageModel_Heinz\Data\building_data_gdb
Downloaded, extracted, and deleted ZIP file for Arkansas to c:\Users\ysurya\Documents\EarthquakeDamageModel_Heinz\Data\building_data_gdb
Downloaded, extracted, and deleted ZIP file for California to c:\Users\ysurya\Documents\EarthquakeDamageModel_Heinz\Data\building_data_gdb
Downloaded, extracted, and deleted ZIP file for Colorado to c:\Users\ysurya\Documents\EarthquakeDamageModel_Heinz\Data\building_data_gdb
Downloaded, extracted, and deleted ZI

# READ BUILDING DATA

In [10]:
def gdb_path_by_state(stateid):
    """Returns the path to the GDB file for a given state ID."""
    cwd = os.getcwd()

    # get parent directory
    parent_dir = os.path.dirname(cwd)
    # get the building data directory
    building_data_directory = os.path.join(parent_dir, 'Data', 'building_data_gdb')
    # find all folder in the building data directory
    folders = [f for f in os.listdir(building_data_directory) if os.path.isdir(os.path.join(building_data_directory, f))]
    # get the folder that ends with stateid
    stateid_dir= [f for f in folders if f.endswith(f'{stateid}')][0]

    return os.path.join(building_data_directory, stateid_dir, f'{stateid}_Structures.gdb')

def get_building_data_csv(stateid):
    """Returns the path to the CSV file for a given state ID."""
    building_data_directory = os.path.join(os.path.dirname(os.getcwd()), 'Data', 'building_data_csv')

    # get the csv file
    return os.path.join(building_data_directory, f'{stateid}_building_data.csv')

In [11]:
# read only the specified columns
def read_cols(path):
    """Read the GDB file and return a GeoDataFrame with specified columns."""
    
    cols = ['BUILD_ID', 'OCC_CLS', 'PRIM_OCC', 'CENSUSCODE', 'LONGITUDE', 'LATITUDE']
    return gpd.read_file(path, columns=cols)
    
# only read specific columns, it can reduce the memory usage and time for each state    

In [24]:
def read_building_data(stateid):
    """Check if the aggregated csv file exists for the given state ID.
    If it does, return the file type and None. Do nothing.
    If it does not, read the GDB file for the state and return the file type and the GeoDataFrame.
    Kwargs:
    stateid -- State ID
    """
    
    # gdb file path
    building_data_directory = gdb_path_by_state(stateid)

    # get the csv file
    csv_path = get_building_data_csv(stateid)
    if os.path.exists(csv_path):
        print(f"Aggregated csv file found for {stateid}")
        return 'csv', None

    else:
        print(f"Reading {building_data_directory}")
        return 'gdb', read_cols(building_data_directory)


In [13]:
states_data = [
    ("Alabama", "AL"), ("Alaska", "AK"), ("Arizona", "AZ"), ("Arkansas", "AR"),
    ("California", "CA"), ("Colorado", "CO"), ("Connecticut", "CT"), ("Delaware", "DE"),
    ("Florida", "FL"), ("Georgia", "GA"), ("Hawaii", "HI"), ("Idaho", "ID"),
    ("Illinois", "IL"), ("Indiana", "IN"), ("Iowa", "IA"), ("Kansas", "KS"),
    ("Kentucky", "KY"), ("Louisiana", "LA"), ("Maine", "ME"), ("Maryland", "MD"),
    ("Massachusetts", "MA"), ("Michigan", "MI"), ("Minnesota", "MN"), ("Mississippi", "MS"),
    ("Missouri", "MO"), ("Montana", "MT"), ("Nebraska", "NE"), ("Nevada", "NV"),
    ("New Hampshire", "NH"), ("New Jersey", "NJ"), ("New Mexico", "NM"), ("New York", "NY"),
    ("North Carolina", "NC"), ("North Dakota", "ND"), ("Ohio", "OH"), ("Oklahoma", "OK"),
    ("Oregon", "OR"), ("Pennsylvania", "PA"), ("Rhode Island", "RI"), ("South Carolina", "SC"),
    ("South Dakota", "SD"), ("Tennessee", "TN"), ("Texas", "TX"), ("Utah", "UT"),
    ("Vermont", "VT"), ("Virginia", "VA"), ("Washington", "WA"), ("West Virginia", "WV"),
    ("Wisconsin", "WI"), ("Wyoming", "WY")
]

# AGGREGATE BUILDING DATA

In [25]:
# function to remap OCC_CLS and PRIM_OCC
def remap_occupancy_classes(gdf):
    """Remap the occupancy classes and primary occupancy from GDB files.
    Kwargs:
    gdf -- GeoDataFrame read from the GDB file"""

    # Define the mapping dictionaries
    building_data = gdf[['BUILD_ID', 'OCC_CLS', 'PRIM_OCC', 'CENSUSCODE', 'LONGITUDE', 'LATITUDE']]
    # mapping the occupancy class
    mapping = {
        'Agriculture':'OTHER', 'Education':'OTHER', 'Residential':'RESIDENTIAL', 'Unclassified':'OTHER',
        'Commercial':'OTHER', 'Government':'OTHER', 'Industrial':'OTHER', 'Utility and Misc':'OTHER',
        'Assembly':'OTHER'
    }
    building_data['OCC_CLS'] = building_data['OCC_CLS'].map(mapping)

    # mapping the primary occupancy
    mapping = {i:'OTHER' for i in building_data['PRIM_OCC'].unique() if i not in ['Single Family Dwelling', 'Multi - Family Dwelling']}
    residential = {'Single Family Dwelling':'SINGLE FAMILY', 'Multi - Family Dwelling':'MULTI FAMILY'}
    mapping.update(residential)
    building_data['PRIM_OCC'] = building_data['PRIM_OCC'].map(mapping)
    return building_data


# function to aggregate the building counts by GEODI, OCC_CLS, PRIM_OCC
def aggregate_building_counts(gdf):
    building_data = remap_occupancy_classes(gdf)
    # group by GEODI, OCC_CLS, PRIM_OCC and sum the counts
    count_building_data = building_data.groupby(['CENSUSCODE', 'OCC_CLS', 'PRIM_OCC']).agg({'BUILD_ID':'count'}).reset_index()
    # rename the columns
    count_building_data = count_building_data.rename(columns={'BUILD_ID':'COUNT'})
    return count_building_data

In [26]:
def pivot_building_data(count_building_data):
    """Pivot the building data to get the count of buildings by OCC_CLS and PRIM_OCC.
    Kwargs:
    count_building_data -- DataFrame with building counts aggregated by CENSUS CODE, OCC_CLS and PRIM_OCC
    """

    df = count_building_data.copy()

    # Create a pivot table
    df_pivot = df.pivot_table(index="CENSUSCODE", columns=["OCC_CLS", "PRIM_OCC"], values="COUNT", aggfunc="sum", fill_value=0)

    # Flatten MultiIndex columns
    df_pivot.columns = [f"{col[0]}_{col[1]}" for col in df_pivot.columns]
    df_pivot = df_pivot.reset_index()
    df_pivot['TOTAL_RESIDENTIAL'] = df_pivot['RESIDENTIAL_MULTI FAMILY'] + df_pivot['RESIDENTIAL_SINGLE FAMILY'] + df_pivot['RESIDENTIAL_OTHER']
    df_pivot['TOTAL_BUILDING'] = df_pivot['TOTAL_RESIDENTIAL'] + df_pivot['OTHER_OTHER']
    return df_pivot

In [27]:
state_name, stateid = ("Alaska", "AL")
print(f"Reading building data for {state_name}")
filetype, gdf = read_building_data(stateid)
if filetype == 'csv':
    pass
elif filetype == 'gdb':
    count_building_data = aggregate_building_counts(gdf)
    df_pivot = pivot_building_data(count_building_data)
    output_path = os.path.join(os.path.dirname(os.getcwd()), 'Data', 'building_data_csv', f"{stateid}_building_data.csv")
    df_pivot.to_csv(output_path, index=False)
    print(f"Saved building data for {state_name} to {output_path}")

Reading building data for Alaska
Reading /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/Data/building_data_gdb/Deliverable202496AL/AL_Structures.gdb


/var/folders/26/f5bmf46n06g2nxwv03lg4yxm0000gn/T/ipykernel_31767/1288599890.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  building_data['OCC_CLS'] = building_data['OCC_CLS'].map(mapping)
/var/folders/26/f5bmf46n06g2nxwv03lg4yxm0000gn/T/ipykernel_31767/1288599890.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  building_data['PRIM_OCC'] = building_data['PRIM_OCC'].map(mapping)


Saved building data for Alaska to /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/Data/building_data_csv/AL_building_data.csv


In [19]:
df_pivot


,CENSUSCODE,OTHER_OTHER,RESIDENTIAL_MULTI FAMILY,RESIDENTIAL_OTHER,RESIDENTIAL_SINGLE FAMILY,TOTAL_RESIDENTIAL,TOTAL_BUILDING
0,06001400100,104,22,10,1162,1194,1298
1,06001400200,46,110,2,538,650,696
2,06001400300,67,416,7,1139,1562,1629
3,06001400400,57,391,3,777,1171,1228
4,06001400500,56,342,6,614,962,1018
...,...,...,...,...,...,...,...
9108,06115040902,759,0,0,0,0,759
9109,06115041001,579,90,360,1807,2257,2836
9110,06115041002,270,155,628,1771,2554,2824
9111,06115041101,373,86,596,903,1585,1958
